In [5]:
!pip install shapely

In [76]:
import json
import csv

In [8]:
neighborhood_geojson = {}
with open('data/neighborhoods.geojson', 'r') as f:
    neighborhood_geojson = json.load(f)

In [42]:
print(neighborhood_geojson.keys())

dict_keys(['type', 'features'])


In [69]:
print(neighborhood_geojson['type'])

FeatureCollection


In [48]:
print(len(neighborhood_geojson['features']))

195


In [12]:
print(neighborhood_geojson['features'][0].keys())

dict_keys(['type', 'properties', 'geometry'])


In [13]:
print(neighborhood_geojson['features'][0]['type'])

Feature


In [14]:
print(neighborhood_geojson['features'][0]['properties'])

{'ntacode': 'BK88', 'shape_area': '54005019.048', 'county_fips': '047', 'ntaname': 'Borough Park', 'shape_leng': '39247.2278309', 'boro_name': 'Brooklyn', 'boro_code': '3'}


In [23]:
print(len(neighborhood_geojson['features'][0]['geometry']))

2


In [27]:
print(neighborhood_geojson['features'][0]['geometry'].keys())

dict_keys(['type', 'coordinates'])


In [28]:
print(neighborhood_geojson['features'][0]['geometry']['type'])

MultiPolygon


In [30]:
print(len(neighborhood_geojson['features'][0]['geometry']['coordinates']))

1


In [31]:
print(len(neighborhood_geojson['features'][0]['geometry']['coordinates'][0]))

1


In [34]:
print(len(neighborhood_geojson['features'][0]['geometry']['coordinates'][0][0]))

130


In [35]:
print(neighborhood_geojson['features'][0]['geometry']['coordinates'][0][0][0])

[-73.97604935657381, 40.631275905646774]


In [90]:
stations = []
with open('data/station_lat_lon.csv', 'r') as f:
    reader = csv.reader(f)
    for line in reader:
        if line[0] == 'stop_name':
            continue
        station = {}
        station['station_name'] = line[0]
        station['lat'] = float(line[1])
        station['lon'] = float(line[2])
        stations.append(station)
stations = stations[1:]

In [91]:
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

match_counter = 0
for feature in neighborhood_geojson['features']:
    polygon = Polygon(feature['geometry']['coordinates'][0][0])
    for station in stations:
        point = Point(station['lon'], station['lat'])
        if polygon.contains(point):
            match_counter += 1
            station['neighborhood'] = feature['properties']['ntaname']
            station['borough'] = feature['properties']['boro_name']
            station['borough_code'] = feature['properties']['boro_code']
print(match_counter, '/', len(stations), 'matches')

445 / 500 matches


In [94]:
stations_to_write = []
stations_recorded = set()
for station in stations:
    if station['station_name'] not in stations_recorded and 'neighborhood' in station:
        stations_to_write.append(station)
        stations_recorded.add(station['station_name'])

for station in stations:
    if station['station_name'] not in stations_recorded and 'neighborhood' not in station:
        stations_to_write.append(station)
        stations_recorded.add(station['station_name'])

for station in stations_to_write:
    del station['lat']
    del station['lon']

In [95]:
with open('data/station_neighborhood_mapping.json', 'w') as f:
    json.dump(stations_to_write, f)
    
with open('data/station_neighborhood_mapping.csv', 'w') as f:
    writer = csv.writer(f)
    header = ['station_name', 'neighborhood', 'borough', 'borough_code']
    writer.writerow(header)
    for station in stations_to_write:
        station_name = station.get('station_name', '')
        neighborhood = station.get('neighborhood', '')
        borough = station.get('borough', '')
        borough_code = station.get('borough_code', '')
        writer.writerow([station_name, neighborhood, borough, borough_code])
    